In [80]:
import glob
import h5py
import numpy as np
import pandas as pd
import os

In [ ]:
import sys
sys.path.append("..")
import features_lib
from conf import Conf

In [26]:
nbats = f['simplified_behaviour']['name'].shape[0]
nbats

9

In [103]:
def construct_bat_name_to_id_map(f, nbats, recorded_bat):
    BAT_NAME_TO_ID = {}

    bat_names = []
    for bat_id in range(nbats):
        bat_name = chr(f[f['simplified_behaviour']['name'][bat_id][0]][0][0])
        if bat_name != recorded_bat:
            bat_names.append(bat_name)

    BAT_NAME_TO_ID = dict(zip(bat_names, range(1, len(bat_names)+1)))
    BAT_NAME_TO_ID[recorded_bat] = 0
    return BAT_NAME_TO_ID
    
def points_to_hd(x1, y1, x2, y2):
    dx = (x2 - x1)
    dy = (y2 - y1)
    with np.errstate(invalid='ignore'):
        hd = np.round((np.arctan2(dy, dx) % (2 * np.pi)) * 180 / np.pi)
    return hd

def split_to_nets(df):
    x1 = df.X
    y1 = df.Y

    NET1_MIN_X = Conf().NET1_MIN_X
    NET1_MAX_X = Conf().NET1_MAX_X
    NET1_MIN_Y = Conf().NET1_MIN_Y
    NET1_MAX_Y = Conf().NET1_MAX_Y
    NET3_MIN_X = Conf().NET3_MIN_X
    NET3_MAX_X = Conf().NET3_MAX_X
    NET3_MIN_Y = Conf().NET3_MIN_Y
    NET3_MAX_Y = Conf().NET3_MAX_Y
    
    net1 = (NET1_MIN_X <= x1) & (x1 <= NET1_MAX_X) & (NET1_MIN_Y <= y1) & (y1 <= NET1_MAX_Y)
    net3 = (NET3_MIN_X <= x1) & (x1 <= NET3_MAX_X) & (NET3_MIN_Y <= y1) & (y1 <= NET3_MAX_Y)

    df['net1'] = net1
    df['net3'] = net3

    df_net1 = df.copy()
    df_net1[~df_net1.net1] = np.nan

    df_net3 = df.copy()
    df_net3[~df_net3.net3] = np.nan

    df_net1.drop(columns=["net1", "net3"], inplace=True)
    df_net3.drop(columns=["net1", "net3"], inplace=True)
    df.drop(columns=["net1", "net3"], inplace=True)

    df_net1.X -= NET1_MIN_X
    df_net1.Y -= NET1_MIN_Y
    df_net3.X -= NET3_MIN_X
    df_net3.Y -= NET3_MIN_Y

    return df, df_net1, df_net3

In [53]:
def build_dataset_inline(bats):
    # expects bat list, which is a list of dataframes, having X,Y,HD(not important), and bat_id
    
    bat0 = bats[0].copy() # x,y,hd of implanted bat
    bat0.drop("bat_id", axis=1,inplace=True)

    for i in bats.keys():
        if i == 0: continue
        # relative position (4. 1 for each bat) - V
        # relative position from nearest bat - V
        # absolute position for other bat (8. X,Y - for each bat) - V
        # bat id for nearest bat (1-4) - V
        # absolute position for nearest bat - V
        # angle toward each of the bats, relative to HD
        
        bat0[features_lib.get_feature_name(i, "D")] = np.sqrt((bats[0].X - bats[i].X) ** 2 + (bats[0].Y - bats[i].Y) ** 2)
        bat0[features_lib.get_feature_name(i, "X")] = bats[i].X
        bat0[features_lib.get_feature_name(i, "Y")] = bats[i].Y
        
        dx = bats[i].X - bat0.X
        dy = bats[i].Y - bat0.Y
        
        bat0[features_lib.get_feature_name(i, "A")] = (bat0.HD - ((np.arctan2(dy, dx) % (2*np.pi)) * 180 / np.pi)) % 360
    
    bat0 = bat0.rename(columns={\
                                "X" : features_lib.get_feature_name(0, "X"),\
                                "Y" : features_lib.get_feature_name(0, "Y"),\
                                "HD" : features_lib.get_feature_name(0, "HD")\
                               })
    return bat0

In [114]:
def get_df(f, recorded_bat):
    IDs, DF1s, DF3s, DFs = [], [], [], []
    nbats = f['simplified_behaviour']['name'].shape[0]

    BAT_NAME_TO_ID = construct_bat_name_to_id_map(f, nbats, recorded_bat)
    print(BAT_NAME_TO_ID)
    for i in range(nbats):
        bat_name = chr(f[f['simplified_behaviour']['name'][i][0]][0][0])
        bat_id = BAT_NAME_TO_ID[bat_name]
        IDs.append(bat_id)
        x1 = f[f["simplified_behaviour"]["pos_on_net"][i][0]][0] * 100
        y1 = f[f["simplified_behaviour"]["pos_on_net"][i][0]][1] * 100
        x2 = f[f["simplified_behaviour"]["pos_on_net"][i][0]][2] * 100
        y2 = f[f["simplified_behaviour"]["pos_on_net"][i][0]][3] * 100
        hd = points_to_hd(x1, y1, x2, y2)

        df = pd.DataFrame({"X": x1, "Y": y1, "HD": hd, "bat_id": bat_id})
        df, df_net1, df_net3 = split_to_nets(df)

        # DFs.append(df)
        DF1s.append(df_net1)
        DF3s.append(df_net3)

    IDs, DF1s, DF3s = list(zip(*sorted(zip(IDs, DF1s, DF3s))))
    DF1s = dict(zip(range(nbats), DF1s))
    DF3s = dict(zip(range(nbats), DF3s))

    df1 = build_dataset_inline(DF1s)
    df3 = build_dataset_inline(DF3s)

    return df1
get_df(f)

TypeError: get_df() missing 1 required positional argument: 'recorded_bat'

In [90]:
x = """4148	P
4148	P
4148	P
87	U
87	U
87	U
87	U
87	U
87	U
87	U
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2329	A1
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6""".split('\n')
d = {}
for i in x:
    k,v = i.split('\t')
    d[k] = v
print(d)

{'4148': 'P', '87': 'U', '2330': 'A', '2329': 'A1', '6627': '7', '2305': 'A2', '7757': '6', '6602': 'S'}


In [115]:
folder_path = r"C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209"
paths = glob.glob(folder_path + "/*.mat")
for path in paths:
    if 'b2305' not in path:
        continue
    print(path)
    f = h5py.File(path, 'r')
    recorded_bat_id = os.path.basename(path).split("_")[0][1:]
    recorded_bat = d[recorded_bat_id][0]
    get_df(f, recorded_bat).to_csv(path.replace(".mat", ".csv"))

C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191220_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191221_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191222_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191223_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191224_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\neural-analysis\inputs\Behavior\all behaviour 20220209\b2305_d191225_simplified_behaviour.mat
{'O': 1, '2': 2, 'P': 3, 'X': 4, 'A': 0}
C:\Users\itayy.WISMAIN\git\n

In [98]:
x = """4148	P
4148	P
4148	P
87	U
87	U
87	U
87	U
87	U
87	U
87	U
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2330	A
2329	A1
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
2329	A1
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
6627	7
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
2305	A2
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
6602	S
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6
7757	6""".split('\n')
d = {}
for i in x:
    k,v = i.split('\t')
    d[k.zfill(4)] = v
print(d)

{'4148': 'P', '0087': 'U', '2330': 'A', '2329': 'A1', '6627': '7', '2305': 'A2', '7757': '6', '6602': 'S'}
